<H1>Capstone Project - Hotel Recommendation Application</H1>

<H3>Applied Data Science Capstone by IBM/Coursera</H3>

### Table of contents <a name="top"></a>
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

These days travelling to different places in the world is faster and easier than before. There are different reasons for a tourist to visit a new city. There are tourists that like to visit different countries to experience different cultures which includes arts, food. There are travelers who love nature and would like to spend most of their time in the nature. Also, couple of friends would like to have parties and nightlife in a city for an occasion. And at the end some cities are perfect for shopping.

As we can see there are different purposes for each tourist to visit a new city. After arriving to a new city, the most important things that tourists need to arrange before arriving is accommodation. It could be time consuming to check all hotels in a city within your budget and find the best one that addresses all your preferences. I called my project as **Hotel Recommendation application**.

In this project, based on visitor's preference and the city’s data, the application is going to recommend a list of hotels that better fit for the visitor.

For an example, if a visitor like to spend more time for visiting arts, so maybe it is better to book a hotel in an area that it is closer to most of the arts venues. Another visitor may prefer a hotel close to area that has more restaurant. So, the application can recommend a different list of hotels.

## Data <a name="data"></a>

Based on my problem explanation, these factors will influence my application:

* **Visitor Preference:** Visitor can an option to choose one of these preferences (Food, Drinks, Shops, Arts, Outdoors, Sights, and TopPicks)
* Number of Venues and Distance from Each Other based on Visitors Preference. This factor is going to create clusters.
* Hotels in the city and their positions by visitor's preference venues

Following data source has been used in this project:
* Number of Venues in each category with their latitude and longitude will be obtained using **Foursquare**
* Map of the Boston City with using Folium library
* Using Geopy Library to retrieve Boston, MA Latitude and Longitude

<H4>Importing All Required Libraries </H4>

In [ ]:
from bs4 import BeautifulSoup  #library web scrapping
import requests     # library for web URL
import pandas as pd # library for data analsysis
import csv          # library for reading CSV file

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import numpy as np

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<H4>Retrieving Boston, MA Latitude and Longitude</H4>

In [ ]:
#use Geopy library for retrieiving Boston Latitude and Longitude

address = 'Boston, MA'

geolocator = Nominatim(user_agent="boston_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Boston City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Boston City are 42.3602534, -71.0582912.


<H4>Foursquare</H4>

Before sending any command to Foursqaure, first we need to setup a connection with Foursquare

In [ ]:
#Define Foursquare Connection

CLIENT_ID = 'JV5WI0EJ5MBUOQQKT1BGKB3ZV2LU5D24S042G04LJJPOWBBF' # your Foursquare ID
CLIENT_SECRET = 'OUQENNESUMAPIIKRTDAPBWMPSRU05AAYN4ACYKERNBHCFJAE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JV5WI0EJ5MBUOQQKT1BGKB3ZV2LU5D24S042G04LJJPOWBBF
CLIENT_SECRET:OUQENNESUMAPIIKRTDAPBWMPSRU05AAYN4ACYKERNBHCFJAE


You can send different commands to Foursquare, but in the project I am going to use two commands:

**Explore:** Returns a list of recommended venues near the current location. You can add the section variable to limit results to venues with the specified category or property. For more info, visit (https://foursquare.com/dev/docs/venues/explore)

**Search:** Returns a list of venues near the current location, matching a search item For more info, visit (https://foursquare.com/dev/docs/venues/search)


In [ ]:
#function sends the explore command to Foursqaure to find all venues nearby a location
def set_foursquare_explore_function(nearby, section, radius, limit):
    explore_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&section={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        NEARBY,
        section,
        radius,
        limit,)
    return explore_url

#function sends a search command to Foursquare to search all venues neaby a location
def set_foursquare_search_function(nearby, search_query, radius, limit):
    #https://foursquare.com/dev/docs/venues/search
    search_url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&near={}&query={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        nearby,
        search_query,
        radius,
        limit,)
    return search_url # display the URL

As the result of the Foursquare is a JSON file, we need to create some function to retrive info from JSON. 

These two function retrieve categries' name based on Foursquare commands (explore or search)

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
#function to extract category name of the venue in Foursquare Search function
def get_search_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = ''
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

After receving JSON data from Foursquare, we need to clean the data and convert the data into Panda Dataframe for analyzing in the future. 


These two functions convert Explore and Search commands' result to Panda Dataframe

In [ ]:
#function returns nearby venues dataframe based on Explore command
def explore_requested_venues(nearby, section, radius, limit):
    venue_url = set_foursquare_explore_function(nearby, section,radius, limit)
    
    venue_results = requests.get(venue_url).json()

    #if a venue has the same category as requested, then it append it to a new list filtered_venues
    venues = venue_results['response']['groups'][0]['items']

    #nearby_venues = json_normalize(filtered_venues) # flatten JSON
    nearby_venues = json_normalize(venues) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.postalCode']
    nearby_venues =nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

    nearby_venues.rename(columns={"postalCode":"ZipCode"}, inplace=True)
    #nearby_venues.head()
    return nearby_venues

#function returns nearby hotels dataframe based on Foursqure Search command
def search_in_foursquare(nearby, search_query, radius, limit):
    search_url = set_foursquare_search_function(nearby, search_query, radius, limit)
    search_results = requests.get(search_url).json()
    results = search_results['response']['venues']

    #nearby_venues = json_normalize(filtered_venues) # flatten JSON
    nearby_results = json_normalize(results) # flatten JSON
    # filter columns
    filtered_search_columns = ['name', 'categories', 'location.lat', 'location.lng','location.postalCode']
    #filtered_hotel_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.postalCode']
    nearby_results =nearby_results.loc[:, filtered_search_columns]

    # filter the category for each row
    nearby_results['categories'] = nearby_results.apply(get_search_category_type, axis=1)

    # clean columns
    nearby_results.columns = [col.split(".")[-1] for col in nearby_venues.columns]

    nearby_results.rename(columns={"name":"Name","lat":"Latitude", "lng":"Longitude","postalCode":"ZipCode"}, inplace=True)
    #nearby_hotels.head()
    return nearby_results

Now that we have the **nearby venues** based on user's prefrence, we can put them into different clusters based on their latitude and longitude

In [ ]:
#function to create clusters based on latitudent and longitued
def create_grouped_clustering(venue_list, number_of_clusters):
    city_grouped_clustering = venue_list.drop(["name","categories","ZipCode"],axis=1)

    # run k-means clustering
    kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(city_grouped_clustering)

    # check cluster labels generated for each row in the dataframe
    #kmeans.labels_[0:10] 
    
    city_grouped_clustering.insert(0, 'Cluster Labels', kmeans.labels_)
    
    return city_grouped_clustering

The last step is putting these clusters into the map. Because of 7 clusters that created, I defined **seven different colors (Red,Blue,Black,Purple,Green,Gray and Brown)** for each cluster.

In [ ]:
#function to create the map
def create_map_clusters(city_latitude, city_longitude, grouped_clustering, section_name):
    map_clusters = folium.Map(location=[city_latitude, city_longitude], zoom_start=13)

# set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    #rainbow = [colors.rgb2hex(i) for i in colors_array]
    rainbow =["Red","Blue","Black","Purple","Green","Gray","Brown"]
    # add markers to the map
    markers_colors = []
    for lat, lon, cluster in zip(grouped_clustering['lat'], grouped_clustering['lng'], grouped_clustering['Cluster Labels']):
        label = folium.Popup(section_name + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=7,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
    return map_clusters

And the purpose of the application is to recommend hotels to the user. The final step is adding hotels to the map of Boston, MA.

In [ ]:
#function to add hotels to the map
def add_hotels_to_map(venues_map,nearby_results):
    for lat, lng, hotel_name, venue_category in zip(nearby_results['Latitude'], nearby_results['Longitude'], nearby_results['Name'], nearby_results['categories']):
        label = '{}, {}'.format(hotel_name, venue_category)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='yellow',
            fill=True,
            fill_color='yellow',
            fill_opacity=0.7,
            parse_html=False).add_to(venues_map)
    final_map_clusters = venues_map
    return final_map_clusters


## Methodology<a name="methodology"></a>


In this application, first I am sending the explore command to Foursquare based on User's preference and then convert the result to a panda dataframe. 

The next step is to find hotels nearby the location that user is going to visit.

After that the application is going to create clusters based on this nearby venues latitude and longitude.

These clusters are ready to be added to the map of the city.

And the final step is adding hotels' location to the map of the city to help visitor to choose a hotel nearby her/his preference.

## Analysis<a name="analysis"></a>

Let's to run this application for different scenarios.

We are going to test this application for Boston, MA. The maximum number of venues are 100. The maximum distance from Boston's latitude and longitude for searching is 10KM. 

The number of defined clusters is 7. 

Therefore, 

**NEARBY = Boston, MA <br>
RADIUS = 10000 <br>
LIMIT = 100 <br>
kcluster = 7**


In [ ]:
#CONSTANT VARIABLES
NEARBY = 'Boston, MA'
RADIUS = 10000
LIMIT = 100
sections = ['Food','Drinks','Shops','Arts','Outdoors','Sights','TopPicks']
search_query = 'Hotel'
# set number of clusters
kclusters = 7

<H3>Scenario 1: Food is the Visitor's Preference</H3>

In [ ]:
#case_number is based on the variable sections
case_number = 0

List 100 Venues that their categories is Food

In [ ]:
nearby_venues= explore_requested_venues(NEARBY, sections[case_number], RADIUS, LIMIT)
nearby_venues.head()

,name,categories,lat,lng,ZipCode
0,Sam LaGrassa's,Sandwich Place,42.356870,-71.059960,02108
1,Yvonne's,New American Restaurant,42.355664,-71.061466,02108
2,haley.henry,Restaurant,42.357574,-71.059495,02108
3,sweetgreen,Salad Place,42.357704,-71.058713,02108
4,Grotto,Italian Restaurant,42.359915,-71.062807,02114


List of 100 hotels near Boston, MA

In [ ]:
nearby_hotels= search_in_foursquare(NEARBY, search_query, RADIUS, LIMIT)
nearby_hotels.head()

,Name,categories,Latitude,Longitude,ZipCode
0,Kimpton Nine Zero Hotel,Hotel,42.357388,-71.060696,02108
1,Godfrey Hotel,Hotel,42.354748,-71.061613,02111
2,Millennium Bostonian Hotel Boston,Hotel,42.361006,-71.056066,02109
3,XV Beacon Hotel,Hotel,42.358217,-71.061900,02108
4,Club Quarters Hotel in Boston,Hotel,42.356241,-71.057579,02110


Creating a Map of Boston with seven clusters of venues with Food category and yellow Venues which are Hotels.

In [ ]:
city_grouped_clustering = create_grouped_clustering(nearby_venues, kclusters)
map_clusters = create_map_clusters(latitude, longitude, city_grouped_clustering,sections[case_number])
final_map = add_hotels_to_map(map_clusters, nearby_hotels)
final_map

<H3>Scenario 2: Drinks is the Visitor's Preference</H3>

In [ ]:
#case_number is based on the variable sections
case_number = 1

List 100 Venues that their categories is Drinks

In [ ]:
nearby_venues= explore_requested_venues(NEARBY, sections[case_number], RADIUS, LIMIT)
nearby_venues.head()

,name,categories,lat,lng,ZipCode
0,Yvonne's,New American Restaurant,42.355664,-71.061466,02108
1,haley.henry,Restaurant,42.357574,-71.059495,02108
2,Trillium Garden On The Greenway,Beer Garden,42.356540,-71.051100,02110
3,The Tip Tap Room,American Restaurant,42.361153,-71.063976,02114
4,O Ya,Sushi Restaurant,42.351502,-71.056763,02111


List of 100 hotels near Boston, MA

In [ ]:
nearby_hotels= search_in_foursquare(NEARBY, search_query, RADIUS, LIMIT)
nearby_hotels.head()

,Name,categories,Latitude,Longitude,ZipCode
0,Kimpton Nine Zero Hotel,Hotel,42.357388,-71.060696,02108
1,Godfrey Hotel,Hotel,42.354748,-71.061613,02111
2,Millennium Bostonian Hotel Boston,Hotel,42.361006,-71.056066,02109
3,XV Beacon Hotel,Hotel,42.358217,-71.061900,02108
4,Club Quarters Hotel in Boston,Hotel,42.356241,-71.057579,02110


Creating a Map of Boston with seven clusters of venues with Drinks category and yellow Venues which are Hotels.

In [ ]:
city_grouped_clustering = create_grouped_clustering(nearby_venues, kclusters)
map_clusters = create_map_clusters(latitude, longitude, city_grouped_clustering,sections[case_number])
final_map = add_hotels_to_map(map_clusters, nearby_hotels)
final_map

<H3>Scenario 3: Shops is the Visitor's Preference</H3>

In [ ]:
#case_number is based on the variable sections
case_number = 2

List 100 Venues that their categories is Shops

In [ ]:
nearby_venues= explore_requested_venues(NEARBY, sections[case_number], RADIUS, LIMIT)
nearby_venues.head()

,name,categories,lat,lng,ZipCode
0,Boston Public Market,Market,42.361950,-71.057466,02108
1,Roche Bros Downtown Crossing,Market,42.355363,-71.060117,02110
2,Newbury Comics,Record Shop,42.360643,-71.054413,02109
3,Whole Foods Market,Grocery Store,42.361732,-71.065874,02114
4,Primark,Clothing Store,42.355538,-71.060095,02110


List of 100 hotels near Boston, MA

In [ ]:
nearby_hotels= search_in_foursquare(NEARBY, search_query, RADIUS, LIMIT)
nearby_hotels.head()

,Name,categories,Latitude,Longitude,ZipCode
0,Kimpton Nine Zero Hotel,Hotel,42.357388,-71.060696,02108
1,Godfrey Hotel,Hotel,42.354748,-71.061613,02111
2,Millennium Bostonian Hotel Boston,Hotel,42.361006,-71.056066,02109
3,XV Beacon Hotel,Hotel,42.358217,-71.061900,02108
4,Club Quarters Hotel in Boston,Hotel,42.356241,-71.057579,02110


Creating a Map of Boston with seven clusters of venues with Shops category and yellow Venues which are Hotels.

In [ ]:
city_grouped_clustering = create_grouped_clustering(nearby_venues, kclusters)
map_clusters = create_map_clusters(latitude, longitude, city_grouped_clustering,sections[case_number])
final_map = add_hotels_to_map(map_clusters, nearby_hotels)
final_map

<H3>Scenario 4: Arts is the Visitor's Preference</H3>

In [ ]:
#case_number is based on the variable sections
case_number = 3

List 100 Venues that their categories is Arts

In [ ]:
nearby_venues= explore_requested_venues(NEARBY, sections[case_number], RADIUS, LIMIT)
nearby_venues.head()

,name,categories,lat,lng,ZipCode
0,Boston Opera House,Opera House,42.354014,-71.062602,02111
1,Improv Asylum Theatre,Comedy Club,42.362931,-71.055404,02113
2,Make Way For Ducklings,Outdoor Sculpture,42.355569,-71.069764,02108
3,Cutler Majestic Theatre,Theater,42.351545,-71.064850,02116
4,Museum of African American History,History Museum,42.360058,-71.065287,02114


List of 100 hotels near Boston, MA

In [ ]:
nearby_hotels= search_in_foursquare(NEARBY, search_query, RADIUS, LIMIT)
nearby_hotels.head()

,Name,categories,Latitude,Longitude,ZipCode
0,Kimpton Nine Zero Hotel,Hotel,42.357388,-71.060696,02108
1,Godfrey Hotel,Hotel,42.354748,-71.061613,02111
2,Millennium Bostonian Hotel Boston,Hotel,42.361006,-71.056066,02109
3,XV Beacon Hotel,Hotel,42.358217,-71.061900,02108
4,Club Quarters Hotel in Boston,Hotel,42.356241,-71.057579,02110


Creating a Map of Boston with seven clusters of venues with Arts category and yellow Venues which are Hotels.

In [ ]:
city_grouped_clustering = create_grouped_clustering(nearby_venues, kclusters)
map_clusters = create_map_clusters(latitude, longitude, city_grouped_clustering,sections[case_number])
final_map = add_hotels_to_map(map_clusters, nearby_hotels)
final_map

<H3>Scenario 5: Outdoors is the Visitor's Preference</H3>

In [ ]:
#case_number is based on the variable sections
case_number = 4

List 100 Venues that their categories is Outdoors

In [ ]:
nearby_venues= explore_requested_venues(NEARBY, sections[case_number], RADIUS, LIMIT)
nearby_venues.head()

,name,categories,lat,lng,ZipCode
0,Boston Common,Park,42.355487,-71.064882,02111
1,The Rose Kennedy Greenway,Park,42.358227,-71.052055,02110
2,North End Park,Park,42.362488,-71.056477,02113
3,Barry's Bootcamp Boston,Gym / Fitness Center,42.354010,-71.059776,02111
4,Equinox Sports Club Boston,Gym,42.353189,-71.063053,02111


List of 100 hotels near Boston, MA

In [ ]:
nearby_hotels= search_in_foursquare(NEARBY, search_query, RADIUS, LIMIT)
nearby_hotels.head()

,Name,categories,Latitude,Longitude,ZipCode
0,Kimpton Nine Zero Hotel,Hotel,42.357388,-71.060696,02108
1,Godfrey Hotel,Hotel,42.354748,-71.061613,02111
2,Millennium Bostonian Hotel Boston,Hotel,42.361006,-71.056066,02109
3,XV Beacon Hotel,Hotel,42.358217,-71.061900,02108
4,Club Quarters Hotel in Boston,Hotel,42.356241,-71.057579,02110


Creating a Map of Boston with seven clusters of venues with Outdoors category and yellow Venues which are Hotels.

In [ ]:
city_grouped_clustering = create_grouped_clustering(nearby_venues, kclusters)

In [ ]:
map_clusters = create_map_clusters(latitude, longitude, city_grouped_clustering,sections[case_number])
final_map = add_hotels_to_map(map_clusters, nearby_hotels)
final_map

<H3>Scenario 6: Sights is the Visitor's Preference</H3>

In [ ]:
#case_number is based on the variable sections
case_number = 5

List 100 Venues that their categories is Sights

In [ ]:
nearby_venues= explore_requested_venues(NEARBY, sections[case_number], RADIUS, LIMIT)
nearby_venues.head()

,name,categories,lat,lng,ZipCode
0,Sam LaGrassa's,Sandwich Place,42.356870,-71.059960,02108
1,Boston Athenaeum,Library,42.357481,-71.061838,02108
2,The Freedom Trail,Historic Site,42.357314,-71.061038,02111
3,Yvonne's,New American Restaurant,42.355664,-71.061466,02108
4,haley.henry,Restaurant,42.357574,-71.059495,02108


List of 100 hotels near Boston, MA

In [ ]:
nearby_hotels= search_in_foursquare(NEARBY, search_query, RADIUS, LIMIT)
nearby_hotels.head()

,Name,categories,Latitude,Longitude,ZipCode
0,Kimpton Nine Zero Hotel,Hotel,42.357388,-71.060696,02108
1,Godfrey Hotel,Hotel,42.354748,-71.061613,02111
2,Millennium Bostonian Hotel Boston,Hotel,42.361006,-71.056066,02109
3,XV Beacon Hotel,Hotel,42.358217,-71.061900,02108
4,Club Quarters Hotel in Boston,Hotel,42.356241,-71.057579,02110


Creating a Map of Boston with seven clusters of venues with Sights category and yellow Venues which are Hotels.

In [ ]:
city_grouped_clustering = create_grouped_clustering(nearby_venues, kclusters)
map_clusters = create_map_clusters(latitude, longitude, city_grouped_clustering,sections[case_number])
final_map = add_hotels_to_map(map_clusters, nearby_hotels)
final_map

<H3>Scenario 7: TopPicks is the Visitor's Preference</H3>

In [ ]:
#case_number is based on the variable sections
case_number = 6

List 100 Venues that their categories is TopPicks

In [ ]:
nearby_venues= explore_requested_venues(NEARBY, sections[case_number], RADIUS, LIMIT)
nearby_venues.head()

,name,categories,lat,lng,ZipCode
0,Iggy's Bread of the World,Bakery,42.393348,-71.148092,02138
1,Tatte Bakery & Cafe,Bakery,42.357904,-71.070439,02114
2,Harvard Book Store,Bookstore,42.372580,-71.116421,02138
3,Boston Common,Park,42.355487,-71.064882,02111
4,Row 34,Seafood Restaurant,42.349570,-71.047215,02210


List of 100 hotels near Boston, MA

In [ ]:
nearby_hotels= search_in_foursquare(NEARBY, search_query, RADIUS, LIMIT)
nearby_hotels.head()

,Name,categories,Latitude,Longitude,ZipCode
0,Kimpton Nine Zero Hotel,Hotel,42.357388,-71.060696,02108
1,Godfrey Hotel,Hotel,42.354748,-71.061613,02111
2,Millennium Bostonian Hotel Boston,Hotel,42.361006,-71.056066,02109
3,XV Beacon Hotel,Hotel,42.358217,-71.061900,02108
4,Club Quarters Hotel in Boston,Hotel,42.356241,-71.057579,02110


Creating a Map of Boston with seven clusters of venues with TopPicks category and yellow Venues which are Hotels.

In [ ]:
city_grouped_clustering = create_grouped_clustering(nearby_venues, kclusters)
map_clusters = create_map_clusters(latitude, longitude, city_grouped_clustering,sections[case_number])
final_map = add_hotels_to_map(map_clusters, nearby_hotels)
final_map

## Result and Discussion<a name="results"></a>

Using Visualization is one of the best way to make a decision. In the project, I used the map of Boston to show the place of visitor's preference venues and the location the hotels near these venues.

<H3> Conclusion</H3><a name="conclusion"></a><sup></a>

The purpose of this project was to recommend hotels to the visitors based on their preference for visiting a city. I ran this application for seven scenarios.